## PoS 블록체인 노드 만들기

In [1]:
# import

import hashlib
import json
from time import time
import random
import requests
from flask import Flask, request, jsonify

## PoS 객체

In [2]:
class Blockchain(object):
    
    def __init__(self, account_name, account_weight):
        self.chain = []   #chain에 여러 block들어온다
        self.current_transaction   # 임시 transaction 넣는다.
        self.nodes = set()   #Node 목록을 보관
        self.miner_wallet = {'account_name' : account_name, 'account_weight' : account_weight}   #지갑 정보 생성
        
        self.new_block(previous_hash='genesis_block', address = account_name)   # genesis_block 생성
        self.account_name = account_name
        self.account_weight = account_weight
        
        
        @staticmethod
        def hash(block):
            block_string = json.dumps(block, sort_keys=True).encode()
            return hashlib.sha256(block_string).hexdigest()    #hash 라이브러리로 sha256 사용
        
        
        @property
        def last_block(self):
            return self.chain[-1]    #체인의 마지막 블록가져오기
        
        def pos(self):
            winner_list = []   #각 노드에서 선정된 사람 리스트
            time.sleep(1)
            my_winner = self.pick_winner(account_name = self.account_name, account_weight = self.account_weight)
            winner_list.append(my_winner) #선정된 사람 리스트에 내 노드 결과 넣기
            time.sleep(1)
            
            
            for target_node in Blockchain.nodes:      # 다른 노드들도 선정된 사람 진행
                print(target_node)
                headers = {'Content-Type' : 'application/json; charset = utf-8'}
                res = requests.get('http://' + target_node  +"/nodes/pick_winner", headers=headers)
                winner_info = json.loads(res.content)     # 근처 노드들 선정 결과 받기
                
                print(winner_info)
                winner_list.append(winner_info['winner'])
            final_winner = max(winner_list, key = winner_list.count) #각 노드의 pos 결과로 가장 많이 선정된 사람을 winner로 선정
            
            print("final_winner selected : ", final_winner)
            return final_winner

        def pick_winner(self, account_name, account_weight):   #"누가 블록 만들래" 사람 뽑기
            candidate_list = []       #PoS 대상자를 뽑을 전체 풀
            
            for w in range(account_weight):  #나의 노드들의 weight 수만큼 추가
                candidate_list.append(account_name)
            
            random.shuffle(candidate_list)        #랜덤으로 섞기
            for x in candidate_list:
                winner = x
                print("선정된 사람은 : ", winner)
                break
            
            return winner       # winner 공개
        
        
        def new_transaction(self, sender, recipient, amount, smart_contract):
            self.current_transaction.append(
                {
                    'sender' : sender, #송신자
                    'recipient' : recipient, #수신자
                    'amount' : amount, #금액
                    'timestmap' : time.time(),
                    'smart_contract' : smart_contract
                }
            )
            return self.last_block['index'] + 1
        
        def new_block(self, previous_hash = None, address = ''):
            block = {
                'index' : len(self.chain)+1,
                'timestamp' : time.time(),
                'transactions' : self.current_transaction,
                'previous_hash' : previous_hash,
                'validator' : address
            }
            block['hash'] = self.hash(block)
            self.current_transaction = []
            self.chain.append(block)
            return block
        
        def valid_chain(self, chain):
            last_block = chain[0]
            current_index = 1
            
            while current_index < len(chain):
                block = chain[current_index]
                print('%s' % last_block)
                print('%s' % block)
                print("\n---------\n")
                if block['previous_hash'] != self.hash(last_block):
                    return False
                last_block = block
                current_index += 1
            return True

In [ ]:
my_ip = 'localhost'
my_port = '5000'
node_identifier = 'node_'+my_port
mine_owner = 'master'
mine_profit = 0.1

blockchain = Blockchain(account_name=mine_owner, account_weight=100)

app = Flask(__name__)


@app.route('/chain', methods=['GET'])
def full_chain():
    print("chain info requested!!")
    response = {
        'chain' : blockchain.chain,
        'length' : len(blockchain.chain),
    }
    return jsonify(response), 200


